### Threshold Definition on Detecting the Stress / Rest State

This will focused on getting the How much the metrics drops from the Rest / Stress state for classifying the stress state

In [1]:
## Importing Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
## Define the path
scipy_rest = os.path.join("Findpeaks_Scipy-Rest-HRV_Metrics.csv")
scipy_stress = os.path.join("Findpeaks_Scipy-Stress-HRV_Metrics.csv")

## Define the cherry pick for scipy
cherry_scipy_rest = os.path.join("Cherry-Findpeaks_Scipy-Rest-HRV_Metrics.csv")
cherry_scipy_stress = os.path.join("Cherry-Findpeaks_Scipy-Stress-HRV_Metrics.csv")

## Define the path for Elgendi
elgendi_rest = os.path.join("Elgendi-Rest-HRV-Metrics.csv")
elgendi_stress = os.path.join("Elgendi-Stress-HRV-Metrics.csv")

## Define the cherry pick for Elgendi
cherry_elgendi_rest = os.path.join("Cherry-Elgendi-Rest-HRV-Metrics.csv")
cherry_elgendi_stress = os.path.join("Cherry-Elgendi-Stress-HRV-Metrics.csv")

## Record rppg Subject
record_rest = os.path.join("hrv_metrics_subject1-rest.csv")
record_stress = os.path.join("hrv_metrics_subject1-stress.csv")

## Read the CSV files
df_scipy_rest = pd.read_csv(cherry_scipy_rest)
df_scipy_stress = pd.read_csv(cherry_scipy_stress)

df_scipy_rest.head(),df_scipy_stress.head()

(  Subject    RPPG PR      GT PR  RPPG MeanNN   GT MeanNN   RPPG SDNN  \
 0     s41  81.428530  98.034000   833.962264  617.429124  349.010749   
 1     s43  68.577569  95.964203  1136.221198  629.002193  718.465097   
 2     s44  67.257494  68.605187  1023.481117  920.673077  372.471869   
 3     s45  55.846749  82.655874  1451.948052  758.109177  927.831207   
 4     s47  83.297722  78.326567   761.050061  775.229978  170.774595   
 
       GT SDNN   RPPG RMSSD    GT RMSSD  RPPG pNN50   GT pNN50   RPPG LF  \
 0   57.246034   489.743928   70.602253   82.075472  17.525773  0.035133   
 1   47.821843   951.342043   67.346224   89.677419   7.368421  0.052770   
 2  183.537949   506.892449  194.124933   91.954023  68.205128  0.054312   
 3  151.220580  1236.128474  229.898547   90.909091  64.556962  0.037576   
 4   81.676803   232.258732   99.105416   75.213675  25.541126  0.052507   
 
       GT LF   RPPG HF     GT HF  RPPG LF/HF  GT LF/HF  
 0  0.026856  0.092660  0.019194    0.379165 

In [3]:
## Drop the ground truth columns
# Assuming the ground truth columns are prefixed with "GT"
gt_columns = ["GT PR", "GT MeanNN", "GT SDNN", "GT RMSSD", "GT pNN50", "GT LF", "GT HF", "GT LF/HF"]

df_scipy_rest = df_scipy_rest.drop(columns=gt_columns, errors='ignore')
df_scipy_stress = df_scipy_stress.drop(columns=gt_columns, errors='ignore')

hrv_metrics = [col for col in df_scipy_rest.columns if col not in ['Subject']]

hrv_merged = pd.merge(df_scipy_rest, df_scipy_stress, on='Subject', suffixes=('_rest', '_stress'))
hrv_merged.head()

metrics_summary = []

# Metrics summary
for metric in hrv_metrics:
    rest_col = f"{metric}_rest"
    stress_col = f"{metric}_stress"
    
    rest_vals = hrv_merged[rest_col][0]
    stress_vals = hrv_merged[stress_col][0]

    # Compute drop percentage
    drop = ((rest_vals - stress_vals) / rest_vals) * 100
    print(f"Metric: {metric}, Rest: {rest_vals}, Stress: {stress_vals}, Drop: {drop.mean()}%")

#     ## Keep only the subjects where the rest > stress (to match the condition)
#     valid_mask = rest_vals > stress_vals
#     filtered_drop = drop[valid_mask]

#     metrics_summary.append({
#         "Metric": metric,
#         "Subjects_Matched": valid_mask.sum(),
#         "Mean_Drop_%": filtered_drop.mean(),
#         "Median_Drop_%": filtered_drop.median(),
#         "Min_Drop_%": filtered_drop.min(),
#         "Max_Drop_%": filtered_drop.max()
#     })

# # Create a DataFrame with the results
# metrics_summary_df = pd.DataFrame(metrics_summary).sort_values(by="Mean_Drop_%", ascending=True)
# metrics_summary_df.reset_index(drop=True, inplace=True)

# metrics_summary_df.head(20)



Metric: RPPG PR, Rest: 81.42853039612433, Stress: 88.20347584923897, Drop: -8.320112643758458%
Metric: RPPG MeanNN, Rest: 833.9622641509434, Stress: 766.4224664224664, Drop: 8.098663528527782%
Metric: RPPG SDNN, Rest: 349.010749213818, Stress: 283.41080173694826, Drop: 18.79596763842954%
Metric: RPPG RMSSD, Rest: 489.7439279883425, Stress: 406.75301611213376, Drop: 16.94577658514312%
Metric: RPPG pNN50, Rest: 82.0754716981132, Stress: 86.32478632478633, Drop: -5.17732586698105%
Metric: RPPG LF, Rest: 0.0351333948065703, Stress: 0.0251320733843029, Drop: 28.466709457854762%
Metric: RPPG HF, Rest: 0.0926599090801163, Stress: 0.0982141413131554, Drop: -5.994212910609218%
Metric: RPPG LF/HF, Rest: 0.3791650041032635, Stress: 0.2558905779583144, Drop: 32.51207912409975%


### Conclussion

Let's settle on the Drop PR rate for rest / stress state is the 8% difference drop from baseline to the stressor.

The formula is going to be like this
$$
\text{PR}_{stress} = \text{PR}_{stress} \times (1 + \frac{\Delta \%}{100})
$$

### Example
- PR<sub>rest</sub> = 75 BPM
- Δ% = −9.71% (i.e., PR increases under stress by 9.71%)

Then, takes a minimum sample later (2 mins), calculate the diff does the value has been greater / equal to the minimum:
$$
\text{PR}_{stress} = 75 \times (1 + \frac{9.71 \%}{100}) = 75 \times 1.0971 = 82.28 \text{ BPM}
$$




### How about ofr multiple metrics?

Define the rule like
```python
stress_detected = (
    ((current_PR - rest_PR) / rest_PR) >= 0.0971 or
    ((rest_RMSSD - current_RMSSD) / rest_RMSSD) >= 0.20 or
    ((current_LF_HF - rest_LF_HF) / rest_LF_HF) >= 1.0
)
```

Or convert into normalized_score, for computing overall index

```python
stress_score = (
    z_score((current_PR - rest_PR) / rest_PR) +
    z_score((rest_RMSSD - current_RMSSD) / rest_RMSSD) +
    z_score((rest_SDNN - current_SDNN) / rest_SDNN)
)

stress_detected = stress_score > threshold
```
